In [1]:
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore")

import logging
import numpy as np

%matplotlib inline

### Setting up random state for reproducibility

In [2]:
RANDOM_STATE = 1234
np.random.seed(RANDOM_STATE)

### Setting up logger

In [3]:
# Logging configuration

logger = logging.getLogger(__name__)

handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)-5s %(levelname)-5s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

### Restrict maximum features

We restrict the maximum number of features a.k.a. our inputs to be 5000. So only top 5000 words will be chosen from IMDB dataset. `load_data` automatically does a 50:50 train test split.

In [4]:
max_features = 5000
max_review_length = 300

In [5]:
from keras.datasets import imdb

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

Using TensorFlow backend.


In [6]:
logger.debug('Length of X_train: %(len)s', {'len': len(x_train)})
logger.debug('Length of X_test: %(len)s', {'len': len(x_test)})

2018-04-18 12:58:31,063 __main__ DEBUG Length of X_train: 25000
2018-04-18 12:58:31,064 __main__ DEBUG Length of X_test: 25000


In [7]:
from keras.preprocessing import sequence

X_train = sequence.pad_sequences(x_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(x_test, maxlen=max_review_length)

In [8]:
logger.debug('Shape of X_train: %(shape)s', {'shape': X_train.shape})
logger.debug('Shape of X_test: %(shape)s', {'shape': X_test.shape})

2018-04-18 12:58:31,854 __main__ DEBUG Shape of X_train: (25000, 300)
2018-04-18 12:58:31,855 __main__ DEBUG Shape of X_test: (25000, 300)


### Simple LSTM

In [9]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(Embedding(max_features, 128, embeddings_initializer='glorot_normal'))
model.add(LSTM(128, dropout = 0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid', kernel_initializer='glorot_normal'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         640000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 771,713
Trainable params: 771,713
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 157s 6ms/step - loss: 0.5051 - acc: 0.7530 - val_loss: 0.4462 - val_acc: 0.7906
Epoch 2/10
25000/25000 [==============================] - 159s 6ms/step - loss: 0.4020 - acc: 0.8234 - val_loss: 0.3865 - val_acc: 0.8318
Epoch 3/10
25000/25000 [==============================] - 152s 6ms/step - loss: 0.3524 - acc: 0.8521 - val_loss: 0.3556 - val_acc: 0.8510
Epoch 4/10
25000/25000 [==============================] - 153s 6ms/step - loss: 0.3184 - acc: 0.8700 - val_loss: 0.3487 - val_acc: 0.8590
Epoch 5/10
25000/25000 [==============================] - 153s 6ms/step - loss: 0.2827 - acc: 0.8853 - val_loss: 0.3221 - val_acc: 0.8686
Epoch 6/10
25000/25000 [==============================] - 165s 7ms/step - loss: 0.2582 - acc: 0.8963 - val_loss: 0.4479 - val_acc: 0.8201
Epoch 7/10
25000/25000 [==============================] - 153s 6ms/step - loss: 0.2435 - acc: 0.9041 - val_loss: 0.3676 - 